# Cross-subject decoding Motor responses 
### (LINDEX v. LMID and RINDEX v. RMID)

#### RH denotes right hand (left hemisphere); LH denotes left hand (right hemisphere)

#### Takuya Ito
#### 08/18/2020

In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools
from importlib import reload
import nibabel as nib
import pathlib

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


## 0.1 Load data

In [4]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



projectdir = '/home/ti61/f_mc1689_1/SRActFlow/'

# Using final partition
networkdef = np.loadtxt(projectdir + 'data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = projectdir + 'data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(list(xticks.keys()))
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

In [5]:
# gsr = True
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = tools.loadMotorResponses(subj,hand='Right')
    data_task_lh[:,:,scount] = tools.loadMotorResponses(subj,hand='Left')
    scount += 1

#### Identify target vertices

In [6]:
tmp = np.squeeze(nib.load(projectdir + 'data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask.dscalar.nii').get_data())
targetmask_rh_ind = np.where(tmp==True)[0]

tmp = np.squeeze(nib.load(projectdir + 'data/results/GroupfMRI/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask.dscalar.nii').get_data())
targetmask_lh_ind = np.where(tmp==True)[0]

## 1.1 Run across subject decoding on hand-specific motor responses

In [14]:
tools = reload(tools)

subjarray = np.hstack((np.arange(len(subjNums)),np.arange(len(subjNums))))
# same for both LH and RH, since mid will be hstacked on ind
labels = np.hstack((np.repeat(1,len(subjNums)),np.repeat(2,len(subjNums))))

svmmat_rh = np.vstack((data_task_rh[targetmask_rh_ind,0,:].T,data_task_rh[targetmask_rh_ind,1,:].T))
svmmat_lh = np.vstack((data_task_lh[targetmask_lh_ind,0,:].T,data_task_lh[targetmask_lh_ind,1,:].T))


#### Now run decoding accuracies (1000 times, with bootstrapping to obtain CIs)
nbootstraps = 1000
for i in range(nbootstraps):
    decoding_rh, rmatch_rh, rmismatch_rh, tmp = tools.randomSplitLOOBaselineCV(1,svmmat_rh,labels,subjarray)
    decoding_lh, rmatch_lh, rmismatch_lh, tmp = tools.randomSplitLOOBaselineCV(1,svmmat_lh,labels,subjarray)
    
    ##### Save out and append file
    # Open/create file
    pathlib.Path(projectdir + 'data/results/MAIN/srModelPredictionAccuracies/test').mkdir(parents=True,exist_ok=True)
    file_rh = open(projectdir + 'data/results/MAIN/srModelPredictionAccuracies/BaselineMotorDecoding/RH_decoding.txt',"a+")
    file_lh = open(projectdir + 'data/results/MAIN/srModelPredictionAccuracies/BaselineMotorDecoding/LH_decoding.txt',"a+")
    # Write out to file
    print(np.mean(decoding_rh),file=file_rh)
    print(np.mean(decoding_lh),file=file_lh)
    # Close file
    file_rh.close()
    file_lh.close()

    if i%25==0:
        print('Permutation', i)
        print('\tDecoding accuracy RH:', np.mean(decoding_rh), '| R-match:', np.mean(rmatch_rh), '| R-mismatch:', np.mean(rmismatch_rh))
        print('\tDecoding accuracy LH:', np.mean(decoding_lh), '| R-match:', np.mean(rmatch_lh), '| R-mismatch:', np.mean(rmismatch_lh))

Permutation 0
	Decoding accuracy RH: 0.7708333333333334 | R-match: 0.08359707389712172 | R-mismatch: -0.08359707389712168
	Decoding accuracy LH: 0.8854166666666666 | R-match: 0.10892391154286311 | R-mismatch: -0.10892391154286311
Permutation 25
	Decoding accuracy RH: 0.8333333333333334 | R-match: 0.09992514181303169 | R-mismatch: -0.09992514181303164
	Decoding accuracy LH: 0.8854166666666666 | R-match: 0.12104601306937202 | R-mismatch: -0.12104601306937206
Permutation 50
	Decoding accuracy RH: 0.84375 | R-match: 0.0931775251657434 | R-mismatch: -0.09317752516574351
	Decoding accuracy LH: 0.9166666666666666 | R-match: 0.12343126363340114 | R-mismatch: -0.12343126363340118
Permutation 75
	Decoding accuracy RH: 0.8541666666666666 | R-match: 0.09009800172259592 | R-mismatch: -0.0900980017225959
	Decoding accuracy LH: 0.9270833333333334 | R-match: 0.13713544358375016 | R-mismatch: -0.1371354435837501
Permutation 100
	Decoding accuracy RH: 0.8229166666666666 | R-match: 0.07349825084807816 | 

## 1.1 Permutation testing - Run across subject decoding on hand-specific motor responses, shuffled labels

In [17]:
tools = reload(tools)

subjarray = np.hstack((np.arange(len(subjNums)),np.arange(len(subjNums))))
# same for both LH and RH, since mid will be hstacked on ind
labels = np.hstack((np.repeat(1,len(subjNums)),np.repeat(2,len(subjNums))))

svmmat_rh = np.vstack((data_task_rh[targetmask_rh_ind,0,:].T,data_task_rh[targetmask_rh_ind,1,:].T))
svmmat_lh = np.vstack((data_task_lh[targetmask_lh_ind,0,:].T,data_task_lh[targetmask_lh_ind,1,:].T))


#### Now run decoding accuracies (1000 times, with bootstrapping to obtain CIs)
nbootstraps = 1000
for i in range(nbootstraps):
    decoding_rh, rmatch_rh, rmismatch_rh, tmp = tools.randomSplitLOOBaselineCV(1,svmmat_rh,labels,subjarray,permutation=True)
    decoding_lh, rmatch_lh, rmismatch_lh, tmp = tools.randomSplitLOOBaselineCV(1,svmmat_lh,labels,subjarray,permutation=True)
    
    ##### Save out and append file
    # Open/create file
    file_rh = open(projectdir + 'data/results/MAIN/srModelPredictionAccuracies/BaselineMotorDecoding/RH_null_decoding.txt',"a+")
    file_lh = open(projectdir + 'data/results/MAIN/srModelPredictionAccuracies/BaselineMotorDecoding/LH_null_decoding.txt',"a+")
    # Write out to file
    print(np.mean(decoding_rh),file=file_rh)
    print(np.mean(decoding_lh),file=file_lh)
    # Close file
    file_rh.close()
    file_lh.close()

    if i%25==0:
        print('Permutation', i)
        print('\tDecoding accuracy RH:', np.mean(decoding_rh), '| R-match:', np.mean(rmatch_rh), '| R-mismatch:', np.mean(rmismatch_rh))
        print('\tDecoding accuracy LH:', np.mean(decoding_lh), '| R-match:', np.mean(rmatch_lh), '| R-mismatch:', np.mean(rmismatch_lh))

Permutation 0
	Decoding accuracy RH: 0.4791666666666667 | R-match: 0.002671307272009659 | R-mismatch: -0.002671307272009749
	Decoding accuracy LH: 0.5104166666666666 | R-match: 0.017101517014077697 | R-mismatch: -0.017101517014077642
Permutation 25
	Decoding accuracy RH: 0.4791666666666667 | R-match: -0.000582831269289002 | R-mismatch: 0.0005828312692890842
	Decoding accuracy LH: 0.5 | R-match: 0.00035018318636792153 | R-mismatch: -0.00035018318636788917
Permutation 50
	Decoding accuracy RH: 0.4166666666666667 | R-match: 0.0006280995325208786 | R-mismatch: -0.0006280995325209328
	Decoding accuracy LH: 0.5208333333333334 | R-match: -0.008034030838194936 | R-mismatch: 0.008034030838194896
Permutation 75
	Decoding accuracy RH: 0.3958333333333333 | R-match: -0.010292512759506106 | R-mismatch: 0.010292512759506019
	Decoding accuracy LH: 0.4479166666666667 | R-match: -0.0009143368271515261 | R-mismatch: 0.0009143368271514908
Permutation 100
	Decoding accuracy RH: 0.5416666666666666 | R-match